In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

**Feedforward_neuralnetwork-pytorch**

**From_scratch**








In [0]:
#DownLoad the Dataset if not  downloaded in LocalComputer,
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

In [0]:
#hyperparameters
input_size =784 #Size of Image is 28X28 ,So input_size is 784 
num_classes =10 
batch_size = 100
learning_rate = 0.0001
num_epochs = 50
hidden_size = 500


In [0]:
# Device configuration for GPU's if present on the Machine
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [0]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [0]:
# Fully connected neural network with one hidden layer
#This is our Model 
#The Linear Layer Computes the y = w.T*x + b

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [0]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)


In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  


In [18]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/50], Step [100/600], Loss: 1.1715
Epoch [1/50], Step [200/600], Loss: 0.6605
Epoch [1/50], Step [300/600], Loss: 0.5089
Epoch [1/50], Step [400/600], Loss: 0.5050
Epoch [1/50], Step [500/600], Loss: 0.3885
Epoch [1/50], Step [600/600], Loss: 0.3320
Epoch [2/50], Step [100/600], Loss: 0.3099
Epoch [2/50], Step [200/600], Loss: 0.2737
Epoch [2/50], Step [300/600], Loss: 0.3084
Epoch [2/50], Step [400/600], Loss: 0.3107
Epoch [2/50], Step [500/600], Loss: 0.2713
Epoch [2/50], Step [600/600], Loss: 0.2670
Epoch [3/50], Step [100/600], Loss: 0.3151
Epoch [3/50], Step [200/600], Loss: 0.3109
Epoch [3/50], Step [300/600], Loss: 0.1948
Epoch [3/50], Step [400/600], Loss: 0.2693
Epoch [3/50], Step [500/600], Loss: 0.1363
Epoch [3/50], Step [600/600], Loss: 0.1859
Epoch [4/50], Step [100/600], Loss: 0.1856
Epoch [4/50], Step [200/600], Loss: 0.2768
Epoch [4/50], Step [300/600], Loss: 0.2080
Epoch [4/50], Step [400/600], Loss: 0.2805
Epoch [4/50], Step [500/600], Loss: 0.1304
Epoch [4/50

In [19]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


Accuracy of the network on the 10000 test images: 98.14 %


In [0]:
# Save the model checkpoint

torch.save(model.state_dict(), 'model.ckpt')